### Webscrapping with Pandas pd.read_html()  + ODBC on SQL Server with pyodbc

#### Source : Wikipedia / IFSC Climbing World Cup (International Federation of Sport Climbing)

 is a series of competition climbing events held during the year at various locations around the world, organized by the IFSC. The first World Cup was held in 1989. For 18 seasons, from 1989 to 2006 World Cups were held under the auspices of the International Council for Competition Climbing which was part of the UIAA. they were called UIAA Climbing World Cups. Since 2007, they have been held under the auspices of the IFSC

At each event, the athletes compete in 3 disciplines: LEAD, SPEED (from 1998), BOULDERING (from 1999)

The number of events varies from year to year, and the winners for each discipline are decided by the points accumulated in the year.

IMPORTANT : #1 Not to be confused with IFSC Climbing World Championships

In [9]:
import numpy as np
import pandas as pd
# Beautifulsoup already installed into Anaconda environment

# Librairy PyODBC for SQL Server connection
import os
import pyodbc
# setup SQL Server connection session
server = '-PCSJN\DATAVIZ'  
database = 'Climbing_Staging'
connection_string= (
                    'DRIVER={SQL Server};SERVER='
                     + server
                     + ';DATABASE='
                     + database 
                     + ';Trusted_Connection=yes'                  
                    )

# Function for injecting dataframe content into SQL Server Table
def template_SQL_insert_values(table_name,dict):
    
    columns = "("
    for k in dict.keys():
        columns += str(k) + ','
    columns = columns[:-1] ### remove last comma
    columns += ")"
    
    req = f"""INSERT INTO {table_name} {columns} VALUES {tuple(dict.values())}"""
    return req

In [10]:
### url
Wiki_IFSC_Climbing_World_Cup = "https://en.wikipedia.org/wiki/IFSC_Climbing_World_Cup"
### récuperer toutes les tables html vu sur le site (url)
Dataset_Wiki_IFSC_Climbing_World_Cup = pd.read_html(Wiki_IFSC_Climbing_World_Cup)
type(Dataset_Wiki_IFSC_Climbing_World_Cup), len(Dataset_Wiki_IFSC_Climbing_World_Cup)

(list, 35)

##### on récupère 1 list de 35 élts

In [3]:
type(Dataset_Wiki_IFSC_Climbing_World_Cup[0])

pandas.core.frame.DataFrame

##### > Chaque élt est un DataFrame Pandas
##### ? Quel est le contenu chacun des 30 dataframes ? Lesquelles nous interessent ?

In [4]:
Dataset_Wiki_IFSC_Climbing_World_Cup[0]

,IFSC Climbing World Cup,IFSC Climbing World Cup.1
0,World Cup series logo,World Cup series logo
1,Seasons,Seasons
2,1989 1990 1991 1992 1993 1994 1995 1996 1997 1...,1989 1990 1991 1992 1993 1994 1995 1996 1997 1...
3,Disciplines,Disciplines
4,Lead Bouldering Speed,Lead Bouldering Speed
5,Most gold medals,Most gold medals
6,Janja Garnbret (43),Janja Garnbret (43)
7,Most titles,Most titles
8,Sandrine Levet (10) Janja Garnbret (10),Sandrine Levet (10) Janja Garnbret (10)


######## [0] is dataset of summarized data about Climbing World Cup dataset > NOT to Discard

In [6]:
Dataset_Wiki_IFSC_Climbing_World_Cup[1]

,0,1
0,NaN,This section needs to be updated. Please help ...


######## [1] has no data > to Discard

In [7]:
Dataset_Wiki_IFSC_Climbing_World_Cup[2]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Ranking,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,Points,1000,805,690,610,545,495,455,415,380,350,325,300,280,260,240


[2] is about World Cup points for each Ranking (Points - Ranking Mapping)

In [8]:
Df_2_Wiki_IFSC_Climbing_World_Cup_Mapping_Points_Ranking = Dataset_Wiki_IFSC_Climbing_World_Cup[2]

In [9]:
Dataset_Wiki_IFSC_Climbing_World_Cup[3]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Ranking,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
1,Points,220,205,185,170,155,145,130,120,105,95,84,73,63,56,48


[3] is the 2nd part of Points - Ranking Mapping

In [10]:
Df_2_Wiki_IFSC_Climbing_World_Cup_Mapping_Points_Ranking_1st = Df_2_Wiki_IFSC_Climbing_World_Cup_Mapping_Points_Ranking
Df_3_Wiki_IFSC_Climbing_World_Cup_Mapping_Points_Ranking_2nd = Dataset_Wiki_IFSC_Climbing_World_Cup[3]

In [11]:
Dataset_Wiki_IFSC_Climbing_World_Cup[4]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Ranking,31,32,33,34,35,36,37,38,39,40,41,42,43,44-45,46
1,Points,42,37,33,30,27,24,21,19,17,15,14,13,12,11,10


[4] is the 3rd part of Points - Ranking Mapping

In [12]:
Df_4_Wiki_IFSC_Climbing_World_Cup_Mapping_Points_Ranking_3rd = Dataset_Wiki_IFSC_Climbing_World_Cup[4]

In [13]:
Dataset_Wiki_IFSC_Climbing_World_Cup[5]

,0,1,2,3,4,5,6,7,8,9
0,Ranking,47-48,49-50,51-53,54-56,57-59,60-63,64-68,69-74,75-80
1,Points,9,8,7,6,5,4,3,2,1


[5] is the 4th part of Points - Ranking Mapping

In [14]:
Df_5_Wiki_IFSC_Climbing_World_Cup_Mapping_Points_Ranking_4th = Dataset_Wiki_IFSC_Climbing_World_Cup[5]

In [15]:
Dataset_Wiki_IFSC_Climbing_World_Cup[6]

,Discipline,Overall score,Overall ranking
0,Lead,665,1
1,Bouldering,470,2
2,Speed,0,—
3,Combined,1135,1


[6] is about overall (points) score and ranking by Discipline

In [16]:
Df_6_Wiki_IFSC_Climbing_World_Cup_Points_Ranking_Overall_byDiscipline = Dataset_Wiki_IFSC_Climbing_World_Cup[6]

In [17]:
Dataset_Wiki_IFSC_Climbing_World_Cup[7]

Discipline                Selected event  \
                     Discipline                Selected event   
0                          Lead                       Villars   
1                          Lead                          Arco   
2                    Bouldering                      Meringen   
3                    Bouldering                        Munich   
4                         Speed                       Wujiang   
5                         Speed                        Xiamen   
6  Product of relative rankings  Product of relative rankings   

                       Rankings           
                        General Relative  
0                             1        1  
1                             1        1  
2                             4        3  
3                             3        2  
4                            27        2  
5                            26        4  
6  Product of relative rankings       48

[7] is about Nb and Name of Discipline(s), General Ranking and Relative Ranking per World Cup Event  

In [18]:
Df_7_Wiki_IFSC_Climbing_World_Cup_Disciplines_Rankings_byEvent = Dataset_Wiki_IFSC_Climbing_World_Cup[7]

In [19]:
Dataset_Wiki_IFSC_Climbing_World_Cup[8]

,Year,Winner,Second,Third
0,1989,Simon Nadin,Didier Raboutou,Jerry Moffatt
1,1990,François Legrand,Jacky Godoffe,Jim Karn
2,1991,François Legrand (2),François Lombard,Yuji Hirayama
3,1992,François Legrand (3),Luca Zardini,Jean-Baptiste Tribout
4,1993,François Legrand (4),François Petit,Yuji Hirayama
5,1994,François Lombard,François Legrand,Jean-Baptiste Tribout
6,1995,François Petit,François Legrand,Arnaud Petit
7,1996,Arnaud Petit,François Petit,Cristian Brenna
8,1997,François Legrand (5),Arnaud Petit,François Petit
9,1998,Yuji Hirayama,Cristian Brenna,Yevgen Kryvosheytsev


[8] is about Men Top 3 winner full names of Climbing World Cup in Disciplin LEAD

In [4]:
Dataset_Wiki_IFSC_Climbing_World_Cup[8].dtypes

Year       int64
Winner    object
Second    object
Third     object
dtype: object

In [54]:
Wiki_IFSC_Climbing_World_Cup_Top3_Men_Lead_Df8 = Dataset_Wiki_IFSC_Climbing_World_Cup[8]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_World_Cup_Top3_Men_Lead_Df8
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Year VARCHAR(100),
                                        Winner VARCHAR(100),
                                        Second VARCHAR(100),
                                        Third VARCHAR(100),                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_World_Cup_Top3_Men_Lead_Df8.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_World_Cup_Top3_Men_Lead_Df8',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [3]:
Dataset_Wiki_IFSC_Climbing_World_Cup[9]

,Year,Winner,Second,Third
0,1999,Christian Core,Serik Kazbekov,Jérôme Meyer
1,2000,Pedro Pons,Salavat Rakhmetov,Daniel Dulac
2,2001,Jérôme Meyer,Mauro Calibani,Daniel Andrada Jimenez
3,2002,Christian Core (2) Jérôme Meyer (2) Malcolm ...,—,—
4,2003,Jérôme Meyer (3),Salavat Rakhmetov,Daniel Dulac
5,2004,Daniel Dulac,Kilian Fischhuber,Jérôme Meyer
6,2005,Kilian Fischhuber,Jérôme Meyer,Daniel Dulac
7,2006,Jérôme Meyer (4),Kilian Fischhuber,Gérome Pouvreau
8,2007,Kilian Fischhuber (2),Dmitri Sarafutdinov,Stephane Julien
9,2008,Kilian Fischhuber (3),David Lama,Dmitri Sarafutdinov


[9] is about Top 3 Men winner full names of Climbing World Cup in Disciplin BOULDERING

In [6]:
Dataset_Wiki_IFSC_Climbing_World_Cup[9].dtypes

Year       int64
Winner    object
Second    object
Third     object
dtype: object

In [55]:
Wiki_IFSC_Climbing_World_Cup_Top3_Men_Bouldering_Df9 = Dataset_Wiki_IFSC_Climbing_World_Cup[9]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_World_Cup_Top3_Men_Bouldering_Df9
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Year VARCHAR(100),
                                        Winner VARCHAR(100),
                                        Second VARCHAR(100),
                                        Third VARCHAR(100),                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_World_Cup_Top3_Men_Bouldering_Df9.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_World_Cup_Top3_Men_Bouldering_Df9',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [ ]:
Dataset_Wiki_IFSC_Climbing_World_Cup[10]

,Year,Winner,Second,Third
0,1998[4],Andrey Vedenmeer,Vladimir Netsvetaev,Alexey Kozlov
1,1999,Tomasz Oleksy,Vladislav Baranov,Vladimir Zakharov
2,2000,Andrey Vedenmeer (2),Iakov Soubbotine,Vladimir Zakharov
3,2001,Maksym Styenkovyy,Alexander Chaoulsky,Alexander Peshekhonov
4,2002,Alexander Peshekhonov,Maksym Styenkovyy,Sergey Sinitsyn
5,2003,Tomasz Oleksy (2),Alexander Peshekhonov,Iakov Soubbotine
6,2004,Sergey Sinitsyn,Evgeny Vaitcekhovsky,Alexander Peshekhonov
7,2005,Evgeny Vaitcekhovsky,Sergey Sinitsyn,Tomasz Oleksy
8,2006,Evgeny Vaitcekhovsky (2),Sergey Sinitsyn,Alexander Peshekhonov
9,2007,Sergey Sinitsyn (2),Evgeny Vaitcekhovsky,Alexander Kosterin


[10] is about Men Top 3 winner full names of Climbing World Cup in Disciplin SPEED

In [9]:
Dataset_Wiki_IFSC_Climbing_World_Cup[10].iloc[0,:]

Year                  1998[4]
Winner       Andrey Vedenmeer
Second    Vladimir Netsvetaev
Third           Alexey Kozlov
Name: 0, dtype: object

In [12]:
# Cleansing Year : delete [ ] 
Dataset_Wiki_IFSC_Climbing_World_Cup[10].iloc[0,0] = 1984
Dataset_Wiki_IFSC_Climbing_World_Cup[10].iloc[0,:]

Year                     1984
Winner       Andrey Vedenmeer
Second    Vladimir Netsvetaev
Third           Alexey Kozlov
Name: 0, dtype: object

In [14]:
Dataset_Wiki_IFSC_Climbing_World_Cup[10].dtypes

Year      object
Winner    object
Second    object
Third     object
dtype: object

In [56]:
Wiki_IFSC_Climbing_World_Cup_Top3_Men_Speed_Df10 = Dataset_Wiki_IFSC_Climbing_World_Cup[10]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_World_Cup_Top3_Men_Speed_Df10
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Year VARCHAR(50),
                                        Winner VARCHAR(100),
                                        Second VARCHAR(100),
                                        Third VARCHAR(100),                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_World_Cup_Top3_Men_Speed_Df10.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_World_Cup_Top3_Men_Speed_Df10',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [26]:
Dataset_Wiki_IFSC_Climbing_World_Cup[11]

,Year,Winner,Second,Third
0,1998[4][8],Yevgen Kryvosheytsev,Tomasz Oleksy,—
1,1999,François Petit,Daniel Andrada Jimenez,Tomasz Oleksy
2,2000,Alexandre Chabot,Salavat Rakhmetov,Serik Kazbekov
3,2001,Alexandre Chabot (2),Serik Kazbekov,Kilian Fischhuber
4,2002,Maksym Styenkovyy,Serik Kazbekov,Kilian Fischhuber
5,2003,Tomasz Oleksy,Evgeny Ovchinnikov,Serik Kazbekov Cédric Lachat
6,2004,Kilian Fischhuber,Flavio Crespi,Gérome Pouvreau
7,2005,Tomasz Oleksy (2),Kilian Fischhuber,Dmitri Sarafutdinov
8,2006,Tomáš Mrázek,David Lama,Kilian Fischhuber
9,2007,Jorg Verhoeven,Tomáš Mrázek,Kilian Fischhuber


[11]  is about Men Top 3 winner full names of Climbing World Cup in Disciplin COMBINED

In [19]:
Dataset_Wiki_IFSC_Climbing_World_Cup[11].iloc[0,:]

Year                1998[4][8]
Winner    Yevgen Kryvosheytsev
Second           Tomasz Oleksy
Third                        —
Name: 0, dtype: object

In [20]:
# Cleansing Year : delete [ ] 
Dataset_Wiki_IFSC_Climbing_World_Cup[11].iloc[0,0] = 1984
Dataset_Wiki_IFSC_Climbing_World_Cup[11].iloc[0,:]

Year                      1984
Winner    Yevgen Kryvosheytsev
Second           Tomasz Oleksy
Third                        —
Name: 0, dtype: object

In [21]:
Dataset_Wiki_IFSC_Climbing_World_Cup[11].dtypes

Year      object
Winner    object
Second    object
Third     object
dtype: object

In [57]:
Wiki_IFSC_Climbing_World_Cup_Top3_Men_Combined_Df11 = Dataset_Wiki_IFSC_Climbing_World_Cup[11]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_World_Cup_Top3_Men_Combined_Df11
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Year VARCHAR(50),
                                        Winner VARCHAR(100),
                                        Second VARCHAR(100),
                                        Third VARCHAR(100),                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_World_Cup_Top3_Men_Combined_Df11.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_World_Cup_Top3_Men_Combined_Df11',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [30]:
Dataset_Wiki_IFSC_Climbing_World_Cup[12]

,Year,Winner,Second,Third
0,1989,Nanette Raybaud [fr],Luisa Iovane [fr],Robyn Erbesfield
1,1990,Isabelle Patissier Lynn Hill,—,Nanette Raybaud [fr]
2,1991,Isabelle Patissier (2),Susi Good,Robyn Erbesfield
3,1992,Robyn Erbesfield,Isabelle Patissier,Lynn Hill
4,1993,Robyn Erbesfield (2),Susi Good,Elena Ovtchinnikova
5,1994,Robyn Erbesfield (3),Isabelle Patissier,Natalie Richer
6,1995,Robyn Erbesfield (4),Laurence Guyon,Liv Sansoz
7,1996,Liv Sansoz,Laurence Guyon,Stéphanie Bodet
8,1997,Muriel Sarkany,Liv Sansoz,Stéphanie Bodet
9,1998,Liv Sansoz (2),Muriel Sarkany,Stéphanie Bodet


 [12] is about Women Top 3 winner full names of Climbing World Cup in Disciplin LEAD

In [13]:
Dataset_Wiki_IFSC_Climbing_World_Cup[12].dtypes

Year       int64
Winner    object
Second    object
Third     object
dtype: object

In [14]:
Wiki_IFSC_Climbing_World_Cup_Top3_Women_Lead_Df12 = Dataset_Wiki_IFSC_Climbing_World_Cup[12]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_World_Cup_Top3_Women_Lead_Df12
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Year VARCHAR(100),
                                        Winner VARCHAR(100),
                                        Second VARCHAR(100),
                                        Third VARCHAR(100),                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_World_Cup_Top3_Women_Lead_Df12.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_World_Cup_Top3_Women_Lead_Df12',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [33]:
Dataset_Wiki_IFSC_Climbing_World_Cup[13]

,Year,Winner,Second,Third
0,1999,Stéphanie Bodet,Elena Choumilova,Sandrine Levet
1,2000,Sandrine Levet,Elena Choumilova,Delphine Martin
2,2001,Sandrine Levet (2),Myriam Motteau,Corinne Theroux
3,2002,Myriam Motteau Nataliya Perlova Lisa Rands,—,—
4,2003,Sandrine Levet (3),Olga Bibik,Nataliya Perlova
5,2004,Sandrine Levet (4),Olga Bibik,Yulia Abramchuk
6,2005,Sandrine Levet (5),Olga Bibik,Yulia Abramchuk
7,2006,Olga Bibik,Juliette Danion,Anna Stöhr
8,2007,Juliette Danion,Olga Shalagina,Natalija Gros
9,2008,Anna Stöhr,Akiyo Noguchi,Yulia Abramchuk


 [13] is about Women Top 3 winner full names of Climbing World Cup in Disciplin BOULDERING

In [15]:
Dataset_Wiki_IFSC_Climbing_World_Cup[13].dtypes

Year       int64
Winner    object
Second    object
Third     object
dtype: object

In [53]:
Wiki_IFSC_Climbing_World_Cup_Top3_Women_Bouldering_Df13 = Dataset_Wiki_IFSC_Climbing_World_Cup[13]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_World_Cup_Top3_Women_Bouldering_Df13
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Year VARCHAR(100),
                                        Winner VARCHAR(100),
                                        Second VARCHAR(100),
                                        Third VARCHAR(100),                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_World_Cup_Top3_Women_Bouldering_Df13.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_World_Cup_Top3_Women_Bouldering_Df13',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [36]:
Dataset_Wiki_IFSC_Climbing_World_Cup[14]

,Year,Winner,Second,Third
0,1998[4],Olga Zakharova,Alena Ostapenko,Nataliya Perlova
1,1999,Olga Zakharova (2),Alena Ostapenko,Zosia Podgorbounskikh
2,2000,Olena Ryepko,Olga Zakharova,Zosia Podgorbounskikh
3,2001,Olga Zakharova (3),Agung Ethi Hendrawati,Zosia Podgorbounskikh
4,2002,Olena Ryepko (2),Maya Piratinskaya,Valentina Yurina
5,2003,Valentina Yurina,Anna Stenkovaya,Olena Ryepko
6,2004,Tatiana Ruyga,Anna Stenkovaya,Agung Ethi Hendrawati
7,2005,Anna Stenkovaya,Valentina Yurina,Olga Evstigneeva
8,2006,Tatiana Ruyga (2),Valentina Yurina,Anna Stenkovaya
9,2007,Tatiana Ruyga (3),Svitlana Tuzhylina,Anna Stenkovaya


[14] is about Women Top 3 winner full names of Climbing World Cup in Disciplin SPEED

In [22]:
Dataset_Wiki_IFSC_Climbing_World_Cup[14].iloc[0,:]

Year               1998[4]
Winner      Olga Zakharova
Second     Alena Ostapenko
Third     Nataliya Perlova
Name: 0, dtype: object

In [23]:
# Cleansing Year : delete [ ] 
Dataset_Wiki_IFSC_Climbing_World_Cup[14].iloc[0,0] = 1984
Dataset_Wiki_IFSC_Climbing_World_Cup[14].iloc[0,:]

Year                  1984
Winner      Olga Zakharova
Second     Alena Ostapenko
Third     Nataliya Perlova
Name: 0, dtype: object

In [24]:
Dataset_Wiki_IFSC_Climbing_World_Cup[14].dtypes

Year      object
Winner    object
Second    object
Third     object
dtype: object

In [51]:
Wiki_IFSC_Climbing_World_Cup_Top3_Women_Speed_Df14 = Dataset_Wiki_IFSC_Climbing_World_Cup[14]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_World_Cup_Top3_Women_Speed_Df14
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Year VARCHAR(50),
                                        Winner VARCHAR(100),
                                        Second VARCHAR(100),
                                        Third VARCHAR(100),                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_World_Cup_Top3_Women_Speed_Df14.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_World_Cup_Top3_Women_Speed_Df14',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [38]:
Dataset_Wiki_IFSC_Climbing_World_Cup[15]

,Year,Winner,Second,Third
0,1998[4][9],Nataliya Perlova,Renata Piszczek,Olena Ostapenko
1,1999,Elena Choumilova,Stéphanie Bodet,Isabelle Bihr
2,2000,Liv Sansoz,Sandrine Levet,Elena Choumilova
3,2001,Sandrine Levet,Martina Čufar,Elena Choumilova Annatina Schultz
4,2002,Sandrine Levet (2),Olga Zakharova,Jenny Lavarda
5,2003,Sandrine Levet (3),Olga Bibik,Barbara Bacher
6,2004,Sandrine Levet (4),Jenny Lavarda,Alexandra Eyer
7,2005,Sandrine Levet (5),Anna Stenkovaya,Jenny Lavarda
8,2006,Angela Eiter,Natalija Gros,Maja Vidmar
9,2007,Natalija Gros,Angela Eiter,Svitlana Tuzhylina


[15] is about Women Top 3 winner full names of Climbing World Cup in Disciplin COMBINED

In [28]:
Dataset_Wiki_IFSC_Climbing_World_Cup[15].dtypes

Year      object
Winner    object
Second    object
Third     object
dtype: object

In [ ]:
Dataset_Wiki_IFSC_Climbing_World_Cup[15].iloc[0,:]

In [25]:
# Cleansing Year : delete [ ] 
Dataset_Wiki_IFSC_Climbing_World_Cup[15].iloc[0,0] = 1984
Dataset_Wiki_IFSC_Climbing_World_Cup[15].iloc[0,:]

Year                  1984
Winner    Nataliya Perlova
Second     Renata Piszczek
Third      Olena Ostapenko
Name: 0, dtype: object

In [34]:
# Dataset_Wiki_IFSC_Climbing_World_Cup[15]['Year'].astype('Int64') # - Refusé par Python TypeError: cannot safely cast non-equivalent object to int64

In [26]:
Dataset_Wiki_IFSC_Climbing_World_Cup[15].dtypes

Year      object
Winner    object
Second    object
Third     object
dtype: object

In [52]:
Wiki_IFSC_Climbing_World_Cup_Top3_Women_Combined_Df15 = Dataset_Wiki_IFSC_Climbing_World_Cup[15]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_World_Cup_Top3_Women_Combined_Df15
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Year VARCHAR(50),
                                        Winner VARCHAR(100),
                                        Second VARCHAR(100),
                                        Third VARCHAR(100),                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_World_Cup_Top3_Women_Combined_Df15.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_World_Cup_Top3_Women_Combined_Df15',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [3]:
Dataset_Wiki_IFSC_Climbing_World_Cup[16]

,Rank,Nation,Gold,Silver,Bronze,Total
0,1,France (FRA),23,14,18,55
1,2,Austria (AUT),14,14,8,36
2,3,Russia (RUS),13,22,15,50
3,4,Japan (JPN),12,8,18,38
4,5,Czech Republic (CZE),9,9,9,27
5,6,Ukraine (UKR),6,6,7,19
6,7,Poland (POL),6,1,4,11
7,8,Spain (ESP),4,7,3,14
8,9,Italy (ITA),4,6,6,16
9,10,Indonesia (INA),3,2,1,6


[16] is about Worldwide Countries' Ranking (Men) in Climbing World Cup (based on medals points) : ?? Medal ?? devrait être Winner, Second and Third rank !!

In [21]:
Dataset_Wiki_IFSC_Climbing_World_Cup[16].dtypes

Rank      object
Nation    object
Gold       int64
Silver     int64
Bronze     int64
Total      int64
dtype: object

In [22]:
Wiki_IFSC_Climbing_World_Cup_Countries_Total_Ranking_Men_Df16 = Dataset_Wiki_IFSC_Climbing_World_Cup[16]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_World_Cup_Countries_Total_Ranking_Men_Df16
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Rank VARCHAR(50),
                                        Nation VARCHAR(50),
                                        Gold INT,
                                        Silver INT,
                                        Bronze INT,
                                        Total INT                                                                  
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_World_Cup_Countries_Total_Ranking_Men_Df16.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_World_Cup_Countries_Total_Ranking_Men_Df16',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [43]:
Dataset_Wiki_IFSC_Climbing_World_Cup[17]

,Rank,Nation,Gold,Silver,Bronze,Total
0,1,France (FRA),22,16,19,57
1,2,Slovenia (SLO),18,13,12,43
2,3,Russia (RUS),13,19,19,51
3,4,Austria (AUT),11,6,11,28
4,5,United States (USA),10,2,9,21
5,6,Japan (JPN),8,15,10,33
6,7,Ukraine (UKR),7,7,6,20
7,8,South Korea (KOR),6,10,2,18
8,9,Belgium (BEL),5,4,3,12
9,10,Poland (POL),5,3,5,13


[17] is about Worldwide Countries' Ranking (Women) in Climbing World Cup (based on medals points) : ?? Medal ?? devrait être Winner, Second and Third rank !!

In [23]:
Dataset_Wiki_IFSC_Climbing_World_Cup[17].dtypes

Rank      object
Nation    object
Gold       int64
Silver     int64
Bronze     int64
Total      int64
dtype: object

In [24]:
Wiki_IFSC_Climbing_World_Cup_Countries_Total_Ranking_Women_Df17 = Dataset_Wiki_IFSC_Climbing_World_Cup[17]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_World_Cup_Countries_Total_Ranking_Women_Df17
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Rank VARCHAR(50),
                                        Nation VARCHAR(50),
                                        Gold INT,
                                        Silver INT,
                                        Bronze INT,
                                        Total INT                                                                  
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_World_Cup_Countries_Total_Ranking_Women_Df17.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_World_Cup_Countries_Total_Ranking_Women_Df17',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [47]:
Dataset_Wiki_IFSC_Climbing_World_Cup[18]

,Rank,Nation,Gold,Silver,Bronze,Total
0,1,France (FRA),178,165,182,525
1,2,Russia (RUS),158,173,163,494
2,3,Austria (AUT),114,84,78,276
3,4,Slovenia (SLO),95,75,58,228
4,5,Japan (JPN),79,109,102,290
5,6,United States (USA),51,55,46,152
6,7,Czech Republic (CZE),39,38,28,105
7,8,Spain (ESP),36,37,23,96
8,9,Poland (POL),36,32,29,97
9,10,Ukraine (UKR),35,32,47,114


[18] is about Worldwide Countries' Ranking (M/W mixed = 'Total') in Climbing World Cup (based on medals points) : ?? Medal ?? devrait être Winner, Second and Third rank !!

In [ ]:
Dataset_Wiki_IFSC_Climbing_World_Cup[18].iloc[0,1] = 'France'
Dataset_Wiki_IFSC_Climbing_World_Cup[18].iloc[1,1] = 'Fédération de Russie'
Dataset_Wiki_IFSC_Climbing_World_Cup[18].iloc[2,1] = 'Autriche'

Dataset_Wiki_IFSC_Climbing_World_Cup[18].iloc[3,1] = 'Slovénie'
Dataset_Wiki_IFSC_Climbing_World_Cup[18].iloc[4,1] = 'Japon'
Dataset_Wiki_IFSC_Climbing_World_Cup[18].iloc[5,1] = 'États-Unis'

Dataset_Wiki_IFSC_Climbing_World_Cup[18].iloc[6,1] = 'République tchèque'
Dataset_Wiki_IFSC_Climbing_World_Cup[18].iloc[7,1] = 'Espagne'
Dataset_Wiki_IFSC_Climbing_World_Cup[18].iloc[8,1] = 'Pologne'

Dataset_Wiki_IFSC_Climbing_World_Cup[18].iloc[9,1] = 'Ukraine'
Dataset_Wiki_IFSC_Climbing_World_Cup[18].iloc[10,1] = 'Corée du Sud'
Dataset_Wiki_IFSC_Climbing_World_Cup[18].iloc[11,1] = 'Chine'

In [12]:
Dataset_Wiki_IFSC_Climbing_World_Cup[18]

,Rank,Nation,Gold,Silver,Bronze,Total
0,1,France,178,165,182,525
1,2,Fédération de Russie,158,173,163,494
2,3,Autriche,114,84,78,276
3,4,Slovénie,95,75,58,228
4,5,Japon,79,109,102,290
5,6,États-Unis,51,55,46,152
6,7,République tchèque,39,38,28,105
7,8,Espagne,36,37,23,96
8,9,Pologne,36,32,29,97
9,10,Ukraine,35,32,47,114


In [25]:
Dataset_Wiki_IFSC_Climbing_World_Cup[18].dtypes

Rank      object
Nation    object
Gold       int64
Silver     int64
Bronze     int64
Total      int64
dtype: object

In [13]:
Wiki_IFSC_Climbing_World_Cup_Countries_Total_Ranking_Df18 = Dataset_Wiki_IFSC_Climbing_World_Cup[18].head(11)

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_World_Cup_Countries_Total_Ranking_Df18
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Rank VARCHAR(50),
                                        Nation VARCHAR(50),
                                        Gold INT,
                                        Silver INT,
                                        Bronze INT,
                                        Total INT                                                                  
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_World_Cup_Countries_Total_Ranking_Df18.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_World_Cup_Countries_Total_Ranking_Df18',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [49]:
Dataset_Wiki_IFSC_Climbing_World_Cup[19]

,vteClimbing competitions,vteClimbing competitions.1
0,Disciplines,Lead climbing (also called sport climbing) Bou...
1,Rankings,Ranking of career IFSC victories by climber
2,Rock,WorldIFSC IFSC Climbing World Cup IFSC Climbin...
3,World,IFSC IFSC Climbing World Cup IFSC Climbing Wor...
4,IFSC,IFSC Climbing World Cup IFSC Climbing World Ch...
5,Multi-sport,Summer Olympics Summer Youth Olympics World Ga...
6,Continental,IFSC African Championships African Youth Champ...
7,IFSC,African Championships African Youth Championsh...
8,Multi-sport,Asian Games European Games
9,National,American Championships Austrian Championships ...


[19] is duplicate from Climbing European Championships Climbing dataset > to discard !!

In [50]:
Dataset_Wiki_IFSC_Climbing_World_Cup[20]

,0,1
0,World,IFSC IFSC Climbing World Cup IFSC Climbing Wor...
1,IFSC,IFSC Climbing World Cup IFSC Climbing World Ch...
2,Multi-sport,Summer Olympics Summer Youth Olympics World Ga...
3,Continental,IFSC African Championships African Youth Champ...
4,IFSC,African Championships African Youth Championsh...
5,Multi-sport,Asian Games European Games
6,National,American Championships Austrian Championships ...
7,Individual,Sportroccia Rock Master Melloblocco Nordic Cha...


[20] is duplicate from Climbing European Championships Climbing dataset > to discard !!

In [51]:
Dataset_Wiki_IFSC_Climbing_World_Cup[21]

,0,1
0,IFSC,IFSC Climbing World Cup IFSC Climbing World Ch...
1,Multi-sport,Summer Olympics Summer Youth Olympics World Ga...


[21] is duplicate from Climbing European Championships Climbing dataset > to discard !!

In [52]:
Dataset_Wiki_IFSC_Climbing_World_Cup[22]

,0,1
0,IFSC,African Championships African Youth Championsh...
1,Multi-sport,Asian Games European Games


[22] is duplicate from Climbing European Championships Climbing dataset > to discard !!

In [55]:
Dataset_Wiki_IFSC_Climbing_World_Cup[23]

,0,1
0,World,UIAA Ice Climbing World Cup UIAA Ice Climbing ...


[23] is duplicate from Climbing European Championships Climbing dataset > to discard !!

In [56]:
Dataset_Wiki_IFSC_Climbing_World_Cup[24]

,0,1
0,International,International Federation of Sport Climbing (IF...
1,Other,USA Climbing


[24] is duplicate from Climbing European Championships Climbing dataset > to discard !!

In [57]:
Dataset_Wiki_IFSC_Climbing_World_Cup[25]

,vteLead climbing at the IFSC Climbing World Cup,vteLead climbing at the IFSC Climbing World Cup.1
0,Seasons,1989 1990 1991 1992 1993 1994 1995 1996 1997 1...


[25] is about Climbing World Cup Seasons for LEAD > Not to discard !!

In [59]:
Df_25_Wiki_IFSC_Climbing_World_Cup_Seasons_Lead = Dataset_Wiki_IFSC_Climbing_World_Cup[25]

In [58]:
Dataset_Wiki_IFSC_Climbing_World_Cup[26]

,vteBouldering at the IFSC Climbing World Cup,vteBouldering at the IFSC Climbing World Cup.1
0,Seasons,1999 2000 2001 2002 2003 2004 2005 2006 2007 2...


[26] is about Climbing World Cup Seasons for BOULDERING > Not to discard !!

In [60]:
Df_26_Wiki_IFSC_Climbing_World_Cup_Seasons_Bouldering = Dataset_Wiki_IFSC_Climbing_World_Cup[26]

In [61]:
Dataset_Wiki_IFSC_Climbing_World_Cup[27]

,vteSpeed climbing at the IFSC Climbing World Cup,vteSpeed climbing at the IFSC Climbing World Cup.1
0,Seasons,1998 1999 2000 2001 2002 2003 2004 2005 2006 2...


[27] is about Climbing World Cup Seasons for SPEED > Not to discard !!

In [63]:
Df_27_Wiki_IFSC_Climbing_World_Cup_Seasons_Speed = Dataset_Wiki_IFSC_Climbing_World_Cup[27]

In [64]:
Dataset_Wiki_IFSC_Climbing_World_Cup[28]

,vteClimbing,vteClimbing.1
0,Types,Rock climbingDisciplines Aid Big wall Multi-pi...
1,Rock climbing,Disciplines Aid Big wall Multi-pitch Boulderin...
2,Disciplines,Aid Big wall Multi-pitch Bouldering Highball C...
3,Rock types,Crack Face Roof Slab
4,Mountaineering,Alpine Via ferrata Himalayan Alpine style Expe...
5,Other,Buildering Canyoning Crane Grass Commercial Pa...
6,Lists,Alpine clubs Climbers Piolet d'Or winners IFSC...
7,Terminology,EquipmentRock Aider Daisy chain Anchor Ascende...
8,Equipment,Rock Aider Daisy chain Anchor Ascender Croll B...
9,Rock,Aider Daisy chain Anchor Ascender Croll Belay ...


[28] is duplicate from Climbing European Championships Climbing dataset > to discard !!

In [65]:
Dataset_Wiki_IFSC_Climbing_World_Cup[29]

,0,1
0,Rock climbing,Disciplines Aid Big wall Multi-pitch Boulderin...
1,Disciplines,Aid Big wall Multi-pitch Bouldering Highball C...
2,Rock types,Crack Face Roof Slab
3,Mountaineering,Alpine Via ferrata Himalayan Alpine style Expe...
4,Other,Buildering Canyoning Crane Grass Commercial Pa...


[29] is duplicate from Climbing European Championships Climbing dataset > to discard !!

In [66]:
Dataset_Wiki_IFSC_Climbing_World_Cup[30]

,0,1
0,Disciplines,Aid Big wall Multi-pitch Bouldering Highball C...
1,Rock types,Crack Face Roof Slab


[30] is duplicate from Climbing European Championships Climbing dataset > to discard !!

In [75]:
Dataset_Wiki_IFSC_Climbing_World_Cup[31]

,0,1
0,Equipment,Rock Aider Daisy chain Anchor Ascender Croll B...
1,Rock,Aider Daisy chain Anchor Ascender Croll Belay ...
2,Ice and snow,Alpenstock Abalakov thread Bottled oxygen Cram...
3,Action,Abseiling Australian Dülfersitz Belaying Front...
4,Other,Ape index Approach shoe Clean Climbing route B...


[31] is duplicate from Climbing European Championships Climbing dataset > to discard !!

In [80]:
Dataset_Wiki_IFSC_Climbing_World_Cup[32]

,0,1
0,Rock,Aider Daisy chain Anchor Ascender Croll Belay ...
1,Ice and snow,Alpenstock Abalakov thread Bottled oxygen Cram...


[32] is duplicate from Climbing European Championships Climbing dataset > to discard !!

In [77]:
Dataset_Wiki_IFSC_Climbing_World_Cup[33]

,0,1
0,Magazines and journals,Alpine Journal Alpinist American Alpine Journa...
1,Non-fiction films,The Alpinist El Capitan The Dawn Wall Free Sol...


[33] is duplicate from Climbing European Championships Climbing dataset > to discard !!

In [78]:
Dataset_Wiki_IFSC_Climbing_World_Cup[34]

,0,1
0,National,Access Fund Alpine Club Alpine Club of Canada ...
1,International,International Federation of Sport Climbing (IF...
